<h2>Segmenting and Clustering Neighborhoods in the city of Toronto, Canada</h2>

In [1]:
#import libraries
import requests
import pandas as pd
import numpy as np

Store the link in a variable, wikipedia_link. Then, using the function get in the request library, download the Wikipedia page with wikipedia_page as argument

In [2]:
# wikipedia link that contains the postal codes
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wikipedia_link)

Use data attribute text to extract XML to a string & assign it to page

In [3]:
page = raw_wikipedia_page.text

Extract the table from the article

In [4]:
start_point = page.find('<table class="wikitable sortable">') 
end_point = page.find("</table>") + 8 # 8 is offset for table tag
table = page[start_point:end_point]


Read the table into a dataframe

In [5]:
df = pd.read_html(table, header = 0)
df = df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Replace "Not Assigned" with NaN

In [6]:
df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop all the rows that do not have an assigned borough

In [7]:
# simply drop whole row with NaN in "Borough" column
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we dropped two rows
df.reset_index(drop=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Replace Nan values in the Neighbourhood column with the Borough

In [8]:
df["Neighbourhood"].replace(np.nan, df["Borough"], inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Merge rows where the Postcode and Borough are the same, separating the neighbourhood name by a comma

In [9]:
# Group By Postcode and Borough and combine the Neighbourhoods on one line
df = df.groupby(['Postcode','Borough'],sort=False,as_index=False).agg(lambda x:', '.join(x))
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Use the shape method to print the number of rows in the dataframe

In [10]:
df.shape

(103, 3)